## 데이터 전처리
 - MEMO
  1. temp.xlsx의 데이터가 3년치로 바꿀 예정
  2. temp.xlsx에서 woba값 없어질 예정

In [1]:
import pandas as pd
import numpy as np

### 1. 크롤링된 데이터 불러오기 및 전처리

#### 1-1. df_midterm (경기결과 + 날씨)

##### 1) 데이터 불러오기

In [2]:
df_midterm = pd.read_excel('temp.xlsx')
df_midterm.tail()

,label,ymd,filed,time,팀0,H/A0,팀1,H/A1,woba0,woba1,평균기온,강수량,평균풍속,상대습도
714,0,20190929,9,14.0,3,0,8,1,0.323111,0.298750,22.7,0.0,2.9,72.3
715,0,20190930,1,18.5,2,1,7,0,0.331000,0.317125,22.4,0.0,5.8,61.0
716,0,20190930,9,18.5,3,0,8,1,0.315333,0.297500,22.1,0.0,3.2,76.6
717,1,20191001,1,18.5,4,0,9,1,0.361750,0.374222,23.1,0.0,6.5,67.1
718,1,20191001,4,18.5,7,1,10,0,0.312625,0.316444,22.4,9.8,6.5,95.3


##### 2) 데이터 전처리

In [3]:
# KIA 이외 경기 제거
idx_num = df_midterm[(df_midterm['팀0'] != 5) & (df_midterm['팀1'] != 5)].index
df_midterm = df_midterm.drop(idx_num)
df_midterm = df_midterm.reset_index(drop=True) # reorder index

print('Length of dataframe KIA: ' + str(len(df_midterm)))
print('Removed data length : ' + str(len(idx_num)))

Length of dataframe KIA: 144
Removed data length : 575


In [4]:
# 팀0 에 KIA (5), 팀1 에 상대팀 번호로 데이터 정리

# 팀0가 KIA인 dataframe
df_midterm0 = df_midterm[df_midterm['팀0'] == 5].copy()

# 팀1이 KIA인 dataframe
df_midterm1 = df_midterm[df_midterm['팀1'] == 5].copy()
# .copy()는 SettingWithCopyWarning을 없애기 위해

df_midterm1.tail()

,label,ymd,filed,time,팀0,H/A0,팀1,H/A1,woba0,woba1,평균기온,강수량,평균풍속,상대습도
127,0,20190906,3,18.5,1,1,5,0,0.357222,0.359750,26.0,0.9,6.1,88.3
134,0,20190914,1,17.0,2,1,5,0,0.352556,0.331500,22.4,0.3,4.3,80.0
135,1,20190917,5,18.5,4,0,5,1,0.349000,0.329333,23.7,0.0,4.3,56.9
140,0,20190923,3,18.5,1,1,5,0,0.361222,0.335875,19.1,0.0,6.1,69.1
143,1,20190928,5,17.0,2,0,5,1,0.260500,0.278667,24.2,0.0,2.5,87.8


In [5]:
# 이어서

# label 값 변경
df_midterm1.loc[:, 'label'] += 1
df_midterm1.loc[df_midterm1['label'] == 2, 'label'] -= 2

# H/A0, H/A1 값 변경\
df_midterm1.loc[:, 'H/A0'] += 1
df_midterm1.loc[df_midterm1['H/A0'] == 2, 'H/A0'] -= 2

df_midterm1.loc[:, 'H/A1'] += 1
df_midterm1.loc[df_midterm1['H/A1'] == 2, 'H/A1'] -= 2

# 팀0, 팀1 변경
df_midterm1.loc[:, '팀1'] = df_midterm1.loc[:, '팀0']
df_midterm1.loc[:, '팀0'] = 5

# df_midterm = df_midterm0 + df_midterm1
df_midterm = df_midterm0.append(df_midterm1)

df_midterm1.tail()

,label,ymd,filed,time,팀0,H/A0,팀1,H/A1,woba0,woba1,평균기온,강수량,평균풍속,상대습도
127,1,20190906,3,18.5,5,0,1,1,0.357222,0.359750,26.0,0.9,6.1,88.3
134,1,20190914,1,17.0,5,0,2,1,0.352556,0.331500,22.4,0.3,4.3,80.0
135,0,20190917,5,18.5,5,1,4,0,0.349000,0.329333,23.7,0.0,4.3,56.9
140,1,20190923,3,18.5,5,0,1,1,0.361222,0.335875,19.1,0.0,6.1,69.1
143,0,20190928,5,17.0,5,1,2,0,0.260500,0.278667,24.2,0.0,2.5,87.8


In [6]:
# 데이터 정리
# H/A1 : H/A0와 대칭되는 값 (ex. H/A0가 0이면 H/A1이 1)
df_midterm = df_midterm.drop(['H/A1'], axis = 1)
df_midterm = df_midterm.sort_index() # index 오름차순으로 정렬

df_midterm.rename(columns = {'ymd':'date'}, inplace = True)

df_midterm.tail()

,label,date,filed,time,팀0,H/A0,팀1,woba0,woba1,평균기온,강수량,평균풍속,상대습도
139,1,20190921,1,17.0,5,0,9,0.271875,0.372000,19.6,0.0,7.2,60.3
140,1,20190923,3,18.5,5,0,1,0.361222,0.335875,19.1,0.0,6.1,69.1
141,0,20190924,5,18.5,5,1,10,0.279556,0.357125,19.3,0.0,3.2,73.8
142,0,20190926,4,18.5,5,0,7,0.277111,0.323375,22.9,0.0,10.1,74.1
143,0,20190928,5,17.0,5,1,2,0.260500,0.278667,24.2,0.0,2.5,87.8


#### 1-2. df_entry (선발라인업)

#####  1) 데이터 불러오기

In [7]:
df_entry = pd.read_excel('player_entry.xlsx')
df_entry.tail()

,Unnamed: 0,team,date,player1,position1,type1,player2,position2,type2,player3,...,type8,player9,position9,type9,pitcher,position,type,result,opponent_p,type.1
4459,4459,18 kt,2018-03-29,오태곤,LF,R,강백호,DH,L,로하스,...,R,정현,SS,R,류희운,P,우투,kt 7:1 SK,문승원,우투
4460,4460,18 kt,2018-03-28,이진영,DH,L,강백호,LF,L,로하스,...,R,정현,SS,R,금민철,P,좌투,kt 8:5 SK,박종훈,우언
4461,4461,18 kt,2018-03-27,심우준,SS,R,박경수,2B,R,로하스,...,L,장성우,C,R,고영표,P,우언,kt 5:8 SK,산체스,우투
4462,4462,18 kt,2018-03-25,정현,SS,R,오정복,DH,R,로하스,...,L,장성우,C,R,주권,P,우투,kt 1:14 KIA,양현종,좌투
4463,4463,18 kt,2018-03-24,심우준,SS,R,이진영,DH,L,로하스,...,L,장성우,C,R,피어밴드,P,좌투,kt 5:4 KIA,헥터,우투


##### 2) 데이터 전처리

In [8]:
df_entry = df_entry.iloc[:,1:-3] # opponent pitcher, index(unnamed:0), result 제거

In [9]:
# dummy data 제거 + 정규경기 이외 경기 제거
# 특징 : pitcher~type column 값이 모두 같은 값을 가짐
idx_num = df_entry[df_entry['pitcher'] == df_entry['position']].index
df_entry = df_entry.drop(idx_num)

print('Length after removing dummy : ' + str(len(df_entry)))
print('Removed data length : ' + str(len(idx_num)))

Length after removing dummy : 4323
Removed data length : 141


In [10]:
# pitcher postion 값 제거 (p로 고정)
df_entry = df_entry.drop('position', axis = 1)

In [11]:
# date 값 변경 (df_midterm의 ymd 형태로)
df_entry['date'] = df_entry['date'].str.replace("-","").astype(int)

In [12]:
# team 이름 변경 (int 형태로)

# df_entry의 team_name 확인
team_list = df_entry['team'].unique().tolist()
print('team_list : ' + str(team_list) + '   , len(' + str(len(team_list)) + ')')

# team_name이 key, team_num이 value인 dictionary  dict_team 생성
# 1~10 : KT, LG, SK, NC, KIA, 삼성, 롯데, 한화, 두산, 키움
# team_name = [KIA, 삼성, 롯데, 두산, Sk, LG, 한화, NC, 키움, KT]
team_num = [5, 6, 7, 9, 3, 2, 8, 4, 10, 1] # team_name 과 순서 동일
dict_team = {}
for i in range(10):
    for idx in range(3):
        idx = i * 3 + idx
        dict_team[team_list[idx]] = team_num[i]

# df_entry의 team이름 변환
for i in team_list:
    df_entry.loc[df_entry['team'] == i, 'team'] = dict_team[i]

team_list : ['20 KIA', '19 KIA', '18 KIA', '20 삼성', '19 삼성', '18 삼성', '20 롯데', '19 롯데', '18 롯데', '20 두산', '19 두산', '18 두산', '20 SK', '19 SK', '18 SK', '20 LG', '19 LG', '18 LG', '20 한화', '19 한화', '18 한화', '20 NC', '19 NC', '18 NC', '20 키움', '19 키움', '18 넥센', '20 KT', '19 KT', '18 kt']   , len(30)


In [13]:
# 팀 별로 dataframe 분할
# df_entry_team : kia 경기 entry
df_entry_team = df_entry[df_entry['team'] == 5].copy()

# df_entry_opponent : kia 를 제외한 경기 entry
df_entry_opponent = df_entry[df_entry['team'] != 5].copy()
entry_opponent_rename_dict = {}
for i in range(10):
    if i != 9:
        entry_opponent_rename_dict['player' + str(i+1)] = 'opponent player' + str(i+1)
        entry_opponent_rename_dict['type' + str(i+1)] = 'opponent type' + str(i+1)
        entry_opponent_rename_dict['position' + str(i+1)] = 'opponent position' + str(i+1)
    else:
        entry_opponent_rename_dict['pitcher'] = 'opponent pitcher'
        entry_opponent_rename_dict['type'] = 'opponent type'
        
df_entry_opponent.rename(columns = entry_opponent_rename_dict, inplace = True)

df_entry_opponent

,team,date,opponent player1,opponent position1,opponent type1,opponent player2,opponent position2,opponent type2,opponent player3,opponent position3,...,opponent position7,opponent type7,opponent player8,opponent position8,opponent type8,opponent player9,opponent position9,opponent type9,opponent pitcher,opponent type
441,6,20201030,박해민,CF,L,김상수,2B,R,구자욱,RF,...,DH,L,최영진,3B,R,강한울,SS,L,원태인,우투
442,6,20201027,박해민,CF,L,최영진,1B,R,구자욱,RF,...,SS,R,김도환,C,R,강한울,2B,L,최채흥,좌투
443,6,20201025,김헌곤,CF,R,구자욱,RF,L,김동엽,LF,...,2B,R,박계범,SS,R,강한울,3B,L,이승민,좌투
444,6,20201024,박해민,CF,L,최영진,3B,R,팔카,DH,...,2B,R,박계범,SS,R,박승규,RF,R,라이블리,우투
445,6,20201021,박해민,CF,L,강한울,SS,L,구자욱,RF,...,1B,R,김지찬,2B,L,김호재,3B,R,원태인,우투
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4459,1,20180329,오태곤,LF,R,강백호,DH,L,로하스,CF,...,2B,R,장성우,C,R,정현,SS,R,류희운,우투
4460,1,20180328,이진영,DH,L,강백호,LF,L,로하스,CF,...,2B,R,장성우,C,R,정현,SS,R,금민철,좌투
4461,1,20180327,심우준,SS,R,박경수,2B,R,로하스,CF,...,LF,R,강백호,DH,L,장성우,C,R,고영표,우언
4462,1,20180325,정현,SS,R,오정복,DH,R,로하스,CF,...,2B,R,강백호,LF,L,장성우,C,R,주권,우투


In [14]:
df_entry = df_entry.reset_index(drop=True) # reorder index
df_entry.tail() # check df_entry

,team,date,player1,position1,type1,player2,position2,type2,player3,position3,...,position7,type7,player8,position8,type8,player9,position9,type9,pitcher,type
4318,1,20180329,오태곤,LF,R,강백호,DH,L,로하스,CF,...,2B,R,장성우,C,R,정현,SS,R,류희운,우투
4319,1,20180328,이진영,DH,L,강백호,LF,L,로하스,CF,...,2B,R,장성우,C,R,정현,SS,R,금민철,좌투
4320,1,20180327,심우준,SS,R,박경수,2B,R,로하스,CF,...,LF,R,강백호,DH,L,장성우,C,R,고영표,우언
4321,1,20180325,정현,SS,R,오정복,DH,R,로하스,CF,...,2B,R,강백호,LF,L,장성우,C,R,주권,우투
4322,1,20180324,심우준,SS,R,이진영,DH,L,로하스,CF,...,2B,R,강백호,LF,L,장성우,C,R,피어밴드,좌투


#### 1-3. df_pitcher (투수 데이터)

##### 1) 데이터 불러오기

##### 2) 데이터 전처리

#### 1-4. df_hitter (타자 데이터)

##### 1) 데이터 불러오기

##### 2) 데이터 전처리

### 2. 데이터 병합
 - pd.merge(df1, df2, on = 'key', how = 'right, left, outer, inner') / key : 기준 column

#### 2-1. df_midterm + df_entry

In [15]:
# KIA entry 병합
df = pd.merge(df_midterm, df_entry_team, on = "date", how = "right")

In [16]:
# opponent team entry 병합
df = pd.merge(df_midterm, df_entry_opponent, on = "date", how = "right")

In [17]:
# 결측치 제거 (None)
#df.isnull().sum() # 결측치 확인
df = df.dropna(axis = 0, how = 'any')

#### 2-2. df_pitcher

#### 2-3. df_hitter

#### 2-4. 확인

In [24]:
attr = df.columns.tolist()
print('Attributes : ', attr)
print('\nAttributes Length : ', len(attr))

Attributes :  ['label', 'date', 'filed', 'time', '팀0', 'H/A0', '팀1', 'woba0', 'woba1', '평균기온', '강수량', '평균풍속', '상대습도', 'team', 'opponent player1', 'opponent position1', 'opponent type1', 'opponent player2', 'opponent position2', 'opponent type2', 'opponent player3', 'opponent position3', 'opponent type3', 'opponent player4', 'opponent position4', 'opponent type4', 'opponent player5', 'opponent position5', 'opponent type5', 'opponent player6', 'opponent position6', 'opponent type6', 'opponent player7', 'opponent position7', 'opponent type7', 'opponent player8', 'opponent position8', 'opponent type8', 'opponent player9', 'opponent position9', 'opponent type9', 'opponent pitcher', 'opponent type']

Attributes Length :  43


In [25]:
df.head()

,label,date,filed,time,팀0,H/A0,팀1,woba0,woba1,평균기온,...,opponent position7,opponent type7,opponent player8,opponent position8,opponent type8,opponent player9,opponent position9,opponent type9,opponent pitcher,opponent type
0,1.0,20190323,5.0,14.0,5.0,1.0,2.0,0.344889,0.354875,4.8,...,C,R,이학주,SS,L,김상수,2B,R,맥과이어,우투
1,1.0,20190323,5.0,14.0,5.0,1.0,2.0,0.344889,0.354875,4.8,...,2B,L,안중열,C,R,신본기,SS,R,레일리,좌투
2,1.0,20190323,5.0,14.0,5.0,1.0,2.0,0.344889,0.354875,4.8,...,2B,L,박세혁,C,L,김재호,SS,R,린드블럼,우투
3,1.0,20190323,5.0,14.0,5.0,1.0,2.0,0.344889,0.354875,4.8,...,CF,R,최항,2B,L,김성현,SS,R,김광현,좌투
4,1.0,20190323,5.0,14.0,5.0,1.0,2.0,0.344889,0.354875,4.8,...,3B,R,유강남,C,R,정주현,2B,R,윌슨,우투


### 3. 병합된 데이터 전처리

In [ ]:
# 결측치 제거 (None)


In [ ]:
# 필요없는 데이터 제거 (axis = 1 : column)
# 중복데이터 (H/A0과 H/A1은 서로 반대)
df_midterm = df_midterm.drop(['팀0', 'H/A1'], axis = 1)

In [ ]:
# hitter, pitcher 수치값 평균

### 4. 데이터 저장

In [ ]:
df.to_excel('kia_database.xlsx')